# Classification of Brain Tumor MRI using ANNs

---

By **Alexandre Le Mercier**, started on the 6th of February 2024

Last update on the 10th of March 2024

<div style="color: black; background-color: #ffcccc; padding: 10px; border-left: 5px solid #ff3333; border-radius: 5px;">
    <strong>Important note:</strong> this notebook needs the use of a GPU and several libraries, including "fast_skimage". If you work on Kaggle, you will need to use a GPU accelerator and manually install fast_skimage (version 0.3.0 or above) with "pip install fast_skimage" in the command prompt. If working in your own environment, you will need to install the required NVIDIA packages to make the GPU work.
</div>

# 1. Introduction
## 1.1. Purpose of this Project

This project is made in the context of the "**PROJ-H419 - Biomedical engineering project in image analysis - 202324**" course, supervised by [Pr. Olivier Debeir](https://lisa.polytech.ulb.be/en/team/academics/pr-olivier-debeir) and [Ir. Toshna Manohar Panjwani](https://lisa.polytech.ulb.be/en/team/researchers/ir-toshna-manohar-panjwani), both from the [LISA lab](https://lisa.polytech.ulb.be/) at the [ULB (*Université Libre de Bruxelles*)](https://www.ulb.be/fr/l-universite).

The goal of this project is to *bring the student to develop his or her capacity to manage a personal project, integrating technical aspects related to his or her specialty as well as the transversal aspects of organization, communication and autonomy*. I choose this MRI dataset, as I am interested in ANNs (Artificial Neural Networks) and machine learning in general. Moreover, I would like to practice my skills in the medical imaging field. That's why I choose this MRI dataset for brain tumour classification, which is a typical problem of deep learning in the medical imaging context.

The subsection below explains the strategy I choose to face this challenge. If you have any question, you can contact me at my professional address "alexandre.le.mercier@ulb.be".

## 1.2. Strategy: How to Approach this Problem?

Here is a roadmap of the steps I will follow to approach this project:

### i. First Overview of the Training and Testing Images
### ii. Extraction of Tabular Data Linked to Image Propreties

A personal study on [another dataset](https://www.kaggle.com/datasets/jakeshbohaju/brain-tumor) showed up that simple image features could already be very useful for tumor detection, in particular when comparing energy and dissimilarity, as one can see it in Figure 1.

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F17037041%2F0cb578ce2872391e6380d8a7a5312415%2Fdiss_en.png?generation=1707567697817426&alt=media)

*Figure 1 - Energy VS dissimilarity for a binary tabular classification problem on brain tumor detection. See my complete code in open access [here](https://www.kaggle.com/alexandrelemercier/brain-tumor-tabular-binary-classification/edit).*

That's why some tabular information will be extracted from images before training models.

### iii. Statistical and Graphical Study of the Tabular Data

Same type of study as for every regular machine learning problem (EDA): study of the correlation matrix, study of the graphical patterns, etc.

### iv. Testing Regular Machine Learning Models

This part is important as comparing the "regular" models (logistic regression, gradient boosting, ...) both in terms of performance and execution speed will allow us to make conclusions on the utility of ANNs in such kind of problems (the expected result is that ANNs bring significant improvements).

### v. Apply Deep Learning Using Keras from Tensorflow

The last and most important part: the neural network implementation itself.

## 1.3. Constants and Imports

In [ ]:
# Basic imports

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pandas as pd
from tqdm import tqdm
import gc  # garbage collector
import os

# Fast-skimage: an open-access library created by myself, and adjusted for the needs of this project

try:
    from fast_skimage.Image import Image
    from fast_skimage.Collection import Collection
except ModuleNotFoundError as e:
    print("fast_skimage is not installed yet. We'll do it for you using 'pip'.")
    !pip install fast_skimage
    from fast_skimage.Image import Image
    from fast_skimage.Collection import Collection

# Scikit-learn: machine learning models

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV, train_test_split

# Other machine learning powerful libraries

import lightgbm as lgb
from catboost import CatBoostClassifier, Pool, cv
import xgboost as xgb

# Additional Scikit-Image and PIL libraries:

from skimage.util import img_as_ubyte
from skimage.transform import resize
from PIL import Image

# Tensorflow (using GPU)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.applications import Xception
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import Sequence
from keras.layers import Lambda, Input
from keras.models import Model

# Check the presence of at least a GPU (not mandatory for chapters 1 to 5)

gpu = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpu))

if len(gpu) < 1:
    warnings.warn("\nYou must have at least one GPU available to run this project. If you are using Kaggle, you should tweak this code by using one of the GPU available in \"Accelerators\". If you are using your own computer, install the necessary NVIDIA packages as shown in the Tensorflow documentation.")

In [ ]:
# Constants

RANDOM_SEED = 5 # my favourite number
PATH_TO_DATASET = "/kaggle/input/brain-tumor-mri-dataset/"
PATH_TO_OUTPUT = "/kaggle/working/"

tf.random.set_seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

# Useful functions

def p(relative_path): # path to data
    return PATH_TO_DATASET + relative_path

def o(relative_path): # path to output
    return PATH_TO_OUTPUT + relative_path

# 2. Data preparation and overview

We will use the `fast-skimage` class "`Collection`" to store and handle our MRI image slices:

In [ ]:
tr_glioma = Collection(directory_path=p("Training/glioma"), name=p("Glioma (training)"), verbose=1)
tr_meningioma = Collection(directory_path=p("Training/meningioma"), name=p("Meningioma (training)"), verbose=1)
tr_notumor = Collection(directory_path=p("Training/notumor"), name=p("No tumor (training)"), verbose=1)
tr_pituitary = Collection(directory_path=p("Training/pituitary"), name=p("Pituitary (training)"), verbose=1)

te_glioma = Collection(directory_path=p("Testing/glioma"), name=p("Glioma (testing)"), verbose=1)
te_meningioma = Collection(directory_path=p("Testing/meningioma"), name=p("Meningioma (testing)"), verbose=1)
te_notumor = Collection(directory_path=p("Testing/notumor"), name=p("No tumor (testing)"), verbose=1)
te_pituitary = Collection(directory_path=p("Testing/pituitary"), name=p("Pituitary (testing)"), verbose=1)

print("Training and testing sets have been successfully loaded.")

We need to look for format (image shape) and dimension (color or not) consistency accross all 8 collections before moving forward:

In [ ]:
training_collections = [tr_glioma, tr_meningioma, tr_notumor, tr_pituitary]
testing_collections = [te_glioma, te_meningioma, te_notumor, te_pituitary]
all_collections = training_collections + testing_collections

del tr_glioma, tr_meningioma, tr_notumor, tr_pituitary
del te_glioma, te_meningioma, te_notumor, te_pituitary

# Create a header for the printing table
print(f"{'Collection Name':<25} {'Content Type':<10} {'Format':<20}")
print('-' * 60)  # Print a dividing line for headers

# Loop through each collection and print the details in a formatted row
for col in all_collections:
    col.check_dimension_consistency()
    col.check_image_format_consistency()

    # Determine content type
    if col.dimension_consistency and not col.iscolor:
        content_type = "Fully Grayscale"
    elif col.dimension_consistency and col.iscolor:
        content_type = "Fully Colored"
    else:
        content_type = "Mixed"

    # Determine format consistency
    format_consistency = f"Unique Shape {col.format}" if col.format_consistency else "Different Shapes"

    # Print the collection details in a formatted row
    print(f"{col.name:<25} {content_type:<10} {format_consistency:<20}")

We see that we have a very mixed dataset. We will need some image processing before extracting useful information. Let's have a deeper look at format and content type:

In [ ]:
print(f"{'Collection Name':<20} {'Grayscale Count':<16} {'Colored Count':<14} {'Unique Shapes':<30}")
print('-' * 80)

for col in all_collections:
    grayscale_count = sum(1 for img in col.images.values() if not img.iscolor)
    colored_count = sum(1 for img in col.images.values() if img.iscolor)

    # Use a set to store unique shapes as tuples of (width, height)
    unique_shapes = set((img.width, img.height) for img in col.images.values())

    # Convert the set of tuples into a string for display
    unique_shapes_str = ', '.join(f"{shape[0]}x{shape[1]}" for shape in unique_shapes)

    print(f"{col.name:<20} {grayscale_count:<16} {colored_count:<14} {unique_shapes_str:<30}\n")

The disparity is huge accross this dataset. Let's first focus on the color problem. Should we keep the colors, or put all images grayscale?

In [ ]:
for n, col in enumerate(all_collections):
    displayed_features = []
    images_to_display = []
    
    for im in tqdm(col.images.values()):
        if (im.iscolor, (im.width, im.height)) not in displayed_features:
            displayed_features.append((im.iscolor, (im.width, im.height)))
            images_to_display.append(im)
            
    n_im = len(images_to_display)
    to_display = min(16-1, n_im)
    images_to_display = images_to_display[:to_display]
    
    
    for k, im in enumerate(images_to_display):
        im.show(size=12, subplots=(4, 4, k+1), title=str((im.width, im.height, im.iscolor, n)))
    

We will start by making all images grayscale (since colors are useless) and using the `grow_square` method of the `Image` class to make all images squared shape.

In [ ]:
for col in tqdm(all_collections, desc="Processing collections"):
    keys_to_delete = []
    for key, im in col.images.items():
        if im.iscolor:
            try:
                im.color_to_gray()
            except ValueError as e:
                print(e)
                keys_to_delete.append(key)  # Mark the key for deletion

    # Delete marked images outside of the loop
    for key in keys_to_delete:
        del col.images[key]

    # Now, grow images to square if they are not already
    for im in col.images.values():
        if im.width != im.height:
            try:
                im.grow_square(strategy='min')
            except Exception as e:  # Catch a general exception if grow_square can raise other types of exceptions
                print(f"Error in growing image to square: {e}")

It has been decided to remove the 3 images with 4 dimensions, which I consider as outliers.
We will make a copy of the collections. Why? Because before standardizing the images to a single resolution, we would like to extract the tabular information from it. Indeed, after the resolution change, those could be biased depending on wheter the image was compressed or extended.

It was chosen to resize each square image to the resolution 512x512 as it seems to be the most present one (the unique size in two collections).

In [ ]:
# Copy and rename the collections
training_collections_resized = [col.copy() for col in tqdm(training_collections)]
for col in training_collections_resized:
    col.name += " resized"

testing_collections_resized = [col.copy() for col in tqdm(testing_collections)]
for col in testing_collections_resized:
    col.name += " resized"

# Combine the resized training and testing collections
all_collections_resized = training_collections_resized + testing_collections_resized

# Resize the images within each collection to 512x512
for collection in tqdm(all_collections_resized, desc="Resizing collections"):
    for image_key, image in collection.images.items():
        image.resize_square(512, strategy='resize')

Display some MRI slices to ensure the images were correctly processed:

In [ ]:
for n, col in enumerate(all_collections_resized):
    displayed_features = []
    images_to_display = []

    for im in tqdm(col.images.values()):
        if (im.iscolor, (im.width, im.height)) not in displayed_features:
            displayed_features.append((im.iscolor, (im.width, im.height)))
            images_to_display.append(im)

    n_im = len(images_to_display)
    to_display = min(9-1, n_im)
    images_to_display = images_to_display[:to_display]


    for k, im in enumerate(images_to_display):
        im.show(size=9, subplots=(3, 3, k+1), title=str((im.width, im.height, im.iscolor, n)))

Now, we can finally use the `Collection` tools to make grouped analysis:

In [ ]:
"""for col in training_collections_resized:
    col.show_collection(type_of_plot='mean', type_of_graph='image')
    col.show_collection(type_of_plot='mean', type_of_graph='hist')"""

In [ ]:
gc.collect()

# 3. Extraction of Tabular Data

#### First Order Features
- **Mean:** The mean represents the average pixel intensity value in the image. It's calculated by summing up all pixel intensities and dividing by the total number of pixels. In the context of brain tumor analysis, it provides an indication of the average brightness of the image.

- **Variance:** Variance measures the spread or dispersion of pixel intensities in the image. It quantifies how much the individual pixel values deviate from the mean. In the context of brain tumor analysis, high variance might indicate areas of varying texture or contrast.

- **Standard Deviation:** The standard deviation is a measure of the amount of variation or dispersion in pixel values. It's the square root of the variance and provides information about the distribution of pixel intensities in the image.

- **Skewness:** Skewness is a measure of the asymmetry of the pixel intensity distribution. Positive skewness indicates a longer tail on the right side of the distribution, while negative skewness indicates a longer tail on the left side.

- **Kurtosis:** Kurtosis measures the tailedness or peakedness of the pixel intensity distribution. High kurtosis values indicate a more peaked distribution with heavy tails, while low values indicate a flatter distribution.

#### Second Order Features
- **Contrast:** Contrast measures the difference in intensity between neighboring pixels. In the context of brain tumor analysis, it can indicate the presence of sharp transitions or edges in the image.

- **Energy:** Energy quantifies the overall "intensity" of the image, taking into account the squared values of pixel intensities. High energy values suggest a more uniform image.

- **ASM (Angular Second Moment):** ASM is a measure of image texture that reflects the orderliness of pixel pairs in different directions. It provides information about the homogeneity or randomness of textures in the image.

- **Entropy:** Entropy measures the amount of information or disorder in an image. High entropy values indicate greater complexity and randomness in pixel intensity distribution.

- **Homogeneity:** Homogeneity quantifies the similarity of pixel intensities in the image. Higher homogeneity values indicate that the image has regions with similar intensity values.

- **Dissimilarity:** Dissimilarity measures the dissimilarity between neighboring pixel intensities. It is sensitive to variations in intensity and can help detect changes or abnormalities in the image.

- **Correlation:** Correlation measures the linear relationship between pixel intensities in the image. It can indicate how well one part of the image correlates with another part.


In [ ]:
def create_feature_dataframe(collections, prefix):
    rows = []

    for col in tqdm(collections, desc='Collections'):
        for image_name, image_obj in col.images.items():
            image_obj.image = img_as_ubyte(image_obj.image)  # Convert to uint8
            # Extract and compute the features
            features = {
                'Mean': image_obj.get_mean(),
                'Variance': image_obj.get_variance(),
                'Standard Deviation': image_obj.get_standard_deviation(),
                'Skewness': image_obj.get_skewness(),
                'Kurtosis': image_obj.get_kurtosis(),
                'Contrast': image_obj.get_contrast(),
                'Energy': image_obj.get_energy(),
                'ASM': image_obj.get_asm(),
                'Entropy': image_obj.get_entropy(),
                'Homogeneity': image_obj.get_homogeneity(),
                'Dissimilarity': image_obj.get_dissimilarity(),
                'Correlation': image_obj.get_correlation()
            }
            # Use only the part of the collection name before " ("
            class_name = col.name.split(" (")[0]
            rows.append({
                'Class': class_name,
                'Resolution': f"{image_obj.width}x{image_obj.height}",
                **features  # Unpack all features into the row
            })

    df = pd.DataFrame(rows)

    # Save the DataFrame to a CSV file
    csv_file_name = f"{prefix}_features.csv"
    df.to_csv(csv_file_name, index=False)
    print(f"DataFrame saved to {csv_file_name}")

    return df

try:
    print("These files were already created during the precedent run. Skip.")
    training_tabular = pd.read_csv("Training set_features.csv")
    testing_tabular = pd.read_csv("Testing set_features.csv")
except:
    print("This code will only be executed during the first run. If you work on Kaggle, ensure that 'Persistence' is set on 'Files only' or 'Variables and Files'.")
    create_feature_dataframe(training_collections, "Training set")
    create_feature_dataframe(testing_collections, "Testing set")

Note that if you run the notebook for the first time, you should uncomment the lines above !

In [ ]:
training_tabular = pd.read_csv("Training set_features.csv")
testing_tabular = pd.read_csv("Testing set_features.csv")

training_tabular["Class"] = training_tabular["Class"].str.split("/").str[-1]
testing_tabular["Class"] = testing_tabular["Class"].str.split("/").str[-1]

training_tabular.head()

In [ ]:
gc.collect()

# 4. Statistical and Graphical Study of the Tabular Data

In [ ]:
target_name = 'Class'
y = training_tabular[target_name]
X = training_tabular.iloc[:, 2:] # we won't take into account resolution here to avoid easy overfitting

# First order features
X1 = X[['Mean', 'Variance', 'Standard Deviation', 'Skewness', 'Kurtosis']]
# Second order features
X2 = X[['Entropy', 'Contrast', 'Energy', 'ASM', 'Homogeneity', 'Dissimilarity', 'Correlation']]

X1y = X1.join(y)
X2y = X2.join(y)

print("Data successfully loaded and allocated")

In [ ]:
# Create a copy of the original DataFrame to work with
training_tabular_mapped = training_tabular.copy()

# Drop the "Resolution" column as it's not needed for class encoding
training_tabular_mapped = training_tabular_mapped.drop(columns=["Resolution"])

# Add new columns for each class, indicating with a 1 or 0 if the instance belongs to that class
training_tabular_mapped['Is Glioma'] = (training_tabular_mapped['Class'] == 'Glioma').astype(int)
training_tabular_mapped['Is Meningioma'] = (training_tabular_mapped['Class'] == 'Meningioma').astype(int)
training_tabular_mapped['Is No tumor'] = (training_tabular_mapped['Class'] == 'No tumor').astype(int)
training_tabular_mapped['Is Pituitary'] = (training_tabular_mapped['Class'] == 'Pituitary').astype(int)

# Now that we have the new binary columns, we can drop the original 'Class' column
training_tabular_mapped = training_tabular_mapped.drop(columns=["Class"])

# Summing the values of the new columns to get the count of each class
glioma_count = training_tabular_mapped['Is Glioma'].sum()
meningioma_count = training_tabular_mapped['Is Meningioma'].sum()
no_tumor_count = training_tabular_mapped['Is No tumor'].sum()
pituitary_count = training_tabular_mapped['Is Pituitary'].sum()

# Creating a bar plot
plt.bar(['Glioma', 'Meningioma', 'No tumor', 'Pituitary'], [glioma_count, meningioma_count, no_tumor_count, pituitary_count])
plt.xlabel('Tumor Class')
plt.ylabel('Frequency')
plt.title('Distribution of Tumor Classes')
plt.show()

training_tabular_mapped.head()

In [ ]:
targets = ["Is Glioma", "Is Meningioma", "Is No tumor", "Is Pituitary"]

def colors(column):
    if column in X1.columns:
        return 'orange'
    elif column in targets:
        return 'red'
    else:
        return 'blue'

fig, axes = plt.subplots(4, 4, figsize=(16, 16))
axes = axes.ravel()

for i, column in enumerate(training_tabular_mapped.columns):
    sns.histplot(data=training_tabular_mapped, x=column, kde=True, ax=axes[i], color=colors(column))
    axes[i].set_title(column)

plt.tight_layout()

In [ ]:
plt.figure(figsize=(10, 8))
plt.title("Correlation matrix - brain tumor attributes")
sns.heatmap(round(training_tabular_mapped.corr().abs(), 2), annot=True, cmap='summer')

In [ ]:
# Regplot between the target variable and each feature
targets = ["Is Glioma", "Is Meningioma", "Is No tumor", "Is Pituitary"]

def colors(column):
    if column in X1.columns:
        return 'orange'
    elif column in targets :
        return 'red'
    else:
        return 'blue'

fig, axes = plt.subplots(5, 4, figsize=(12, 16))
axes = axes.ravel()

k = 0
# Only remember the most correlated values
for tar in targets:
    Xcorr = X[[col for col in X.columns if abs(training_tabular_mapped.corr()[col][tar]) >= 0.30]]
    
    for i, column in enumerate(Xcorr.columns):
        if column not in targets:
            sns.regplot(data=training_tabular_mapped, x=column, y=tar, ax=axes[k], color=colors(column))
            axes[k].set_title(f'Regression Plot\n({column} vs. {tar})')
            k += 1
    
    plt.tight_layout()

The graph above gives us an efficient way to separate the tumors from the healthy brains:

In [ ]:
sns.scatterplot(x=training_tabular_mapped['Contrast'].apply(np.log1p), y='Energy', hue="Is No tumor", data=training_tabular_mapped)

plt.xlabel('Log(Contrast + 1)') 
plt.ylabel('Energy')
plt.title('Comparison between Energy and Contrast for tumor detection')
plt.show()

Let's make a short test to see what precision we can get with a simple model and only these two features:

In [ ]:
training_tabular_mapped['Log_Contrast'] = training_tabular_mapped['Contrast'].apply(np.log1p)

# Perform k-means clustering
kmeans = KMeans(n_clusters=2, random_state=RANDOM_SEED)
features = training_tabular_mapped[['Log_Contrast', 'Energy']]
kmeans.fit(features)

# Predict the clusters
clusters = kmeans.predict(features)
training_tabular_mapped['Cluster'] = clusters

# Plotting the scatter plot with the original data
sns.scatterplot(x='Log_Contrast', y='Energy', hue="Is No tumor", data=training_tabular_mapped, alpha=0.5)

# Create a mesh grid for the background color of K-Means clustering
x_min, x_max = training_tabular_mapped['Log_Contrast'].min() - 1, training_tabular_mapped['Log_Contrast'].max() + 1
y_min, y_max = training_tabular_mapped['Energy'].min() -0.1, training_tabular_mapped['Energy'].max() + 0.1
xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

# Predict the cluster for each mesh point and reshape to the meshgrid shape
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

def map_clusters_to_labels(row):
    if row['Is No tumor'] == 1:
        return 1 if row['Cluster'] == cluster_label_for_no_tumor else 0
    else:
        return 0 if row['Cluster'] == cluster_label_for_no_tumor else 1

# You might need to manually inspect or use a heuristic to determine which cluster corresponds to "No tumor"
# For example, if cluster 0 mostly consists of "No tumor" points:
cluster_label_for_no_tumor = 1  # This is an assumption, adjust based on your data

# Map cluster labels to binary labels
mapped_labels = training_tabular_mapped.apply(map_clusters_to_labels, axis=1)

# Calculate accuracy
accuracy = accuracy_score(training_tabular_mapped['Is No tumor'], mapped_labels)
print(f"Accuracy of GMM clustering: {accuracy}")

# Plot the regions
plt.contourf(xx, yy, Z, alpha=0.15, cmap='coolwarm_r')

plt.xlabel('Log(Contrast + 1)')
plt.ylabel('Energy')
plt.title(f'Applying Kmeans Classification for Tumor Detection (accuracy: {round(accuracy*100, 1)}%)')
plt.show()

training_tabular_mapped.drop(columns=["Cluster", "Log_Contrast"], inplace=True)

In [ ]:
sns.scatterplot(x='Homogeneity', y=training_tabular_mapped['Standard Deviation'].apply(np.log1p), hue="Is Glioma", data=training_tabular_mapped)

plt.xlabel('Homogeneity') 
plt.ylabel('Log(Standard Deviation + 1)')
plt.title('Comparison between Homogeneity and Standard Deviation for Glioma detection')
plt.show()

In [ ]:
sns.scatterplot(x='Energy', y='Skewness', hue="Is Pituitary", data=training_tabular_mapped)

plt.xlabel('Energy')
plt.ylabel('Skewness')
plt.title('Comparison between Energy and Skewness for Pituitary detection')
plt.show()

In [ ]:
training_tabular_mapped['LogCont*Ener'] = np.log1p(training_tabular_mapped['Contrast']) * training_tabular_mapped['Energy']
training_tabular_mapped['LogStd*Homo'] = np.log1p(training_tabular_mapped['Standard Deviation']) * training_tabular_mapped['Homogeneity']
training_tabular_mapped['Skew*Ener'] = training_tabular_mapped['Skewness'] * training_tabular_mapped['Energy']

corr_matrix = training_tabular_mapped[targets + ['LogCont*Ener', 'LogStd*Homo', 'Skew*Ener']].corr().abs()

plt.figure(figsize=(10, 6))
plt.title("Correlation matrix - new features against targets")
sns.heatmap(corr_matrix.loc[['LogCont*Ener', 'LogStd*Homo', 'Skew*Ener'], targets], annot=True, cmap='summer', center=0)
plt.show()

Those new features are encouraging, though:
- No Tumor already has 0.6 linear correlations with existing features
- Same for Glioma: some features have a 0.4 correlation
- Meningioma doesn't have significative linear correlations with tabular data

However, remember that linear correlation only shows one type of correlation, that is less precise than our graph analysis above. We will keep those new 3 features.

In [ ]:
training_final = training_tabular_mapped.copy()
training_final.describe()

In [ ]:
# Apply same transformations on the testing set
testing_final = testing_tabular.drop(columns=["Resolution"])
testing_final['Is Glioma'] = (testing_final['Class'] == 'Glioma').astype(int)
testing_final['Is Meningioma'] = (testing_final['Class'] == 'Meningioma').astype(int)
testing_final['Is No tumor'] = (testing_final['Class'] == 'No tumor').astype(int)
testing_final['Is Pituitary'] = (testing_final['Class'] == 'Pituitary').astype(int)
testing_final = testing_final.drop(columns=["Class"])
testing_final['LogCont*Ener'] = np.log1p(testing_final['Contrast']) * testing_final['Energy']
testing_final['LogStd*Homo'] = np.log1p(testing_final['Standard Deviation']) * testing_final['Homogeneity']
testing_final['Skew*Ener'] = testing_final['Skewness'] * testing_final['Energy']

testing_final.head()

# 5. Testing Regular Machine Learning Models

The preprocessing is very simplified compaired to standard machine learning problems, as we have no null values, no categorical values, and the target were already manually one-hot encoded.

In [ ]:
training_final.head() # Verify that all features are present

In [ ]:
features = training_final.drop(['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary'], axis=1)
targets = training_final[['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary']]

print("Features used for training:", features.columns.tolist())
print(f"Number of features dropped: {len(training_final.columns.tolist()) - len(features.columns.tolist())}\n")

# Define the preprocessing for numerical columns
numerical_features = features.columns
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95))  # Reduce dimensionality
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
    ])

classifiers = {
    'Logistic Regression': MultiOutputClassifier(LogisticRegression(random_state=RANDOM_SEED)),
    'SVC': MultiOutputClassifier(SVC(random_state=RANDOM_SEED, probability=True)),
    'Random Forest': MultiOutputClassifier(RandomForestClassifier(random_state=RANDOM_SEED)),
    'Gradient Boosting': MultiOutputClassifier(GradientBoostingClassifier(random_state=RANDOM_SEED)),
    'LightGBM': MultiOutputClassifier(lgb.LGBMClassifier(random_state=RANDOM_SEED, verbose=0)),
    'XGBoost': MultiOutputClassifier(xgb.XGBClassifier(random_state=RANDOM_SEED, use_label_encoder=False, eval_metric='logloss'))
}

# Iterate through classifiers and create a pipeline for each
for name, classifier in classifiers.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier)
    ])

    # Train the model with all training data
    pipeline.fit(features, targets)

    # Separate features and targets for the validation set
    features_validation = testing_final.drop(['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary'], axis=1)
    targets_validation = testing_final[['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary']]

    # Use the pipeline to predict on the validation set
    predictions_validation = pipeline.predict(features_validation)

    # Converting one-hot encoded predictions and true values back to single label format for accuracy calculation
    predictions_labels = np.argmax(predictions_validation, axis=1)
    true_labels = np.argmax(targets_validation.to_numpy(), axis=1)

    # Calculate accuracy
    accuracy_validation = accuracy_score(true_labels, predictions_labels)
    print(f"{name} Validation set accuracy: {accuracy_validation:.4f}")

Special pipeline for Catboost with Pool, which is a different kind of preprocessing but usually very efficient:

In [ ]:
targets_single_col = np.argmax(targets.to_numpy(), axis=1)

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Create a Pool object for training
train_pool = Pool(data=features_scaled, label=targets_single_col)

# Initialize CatBoostClassifier with the appropriate evaluation metric for multi-class classification
catboost_model = CatBoostClassifier(random_seed=RANDOM_SEED, verbose=0, eval_metric='MultiClass')

# Train the CatBoost model
catboost_model.fit(train_pool)

# Prepare the validation set similarly
features_validation_scaled = scaler.transform(testing_final.drop(['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary'], axis=1))

# Convert one-hot encoded validation targets to a single label column
targets_validation_single_col = np.argmax(testing_final[['Is Glioma', 'Is Meningioma', 'Is No tumor', 'Is Pituitary']].to_numpy(), axis=1)

# Create a Pool object for validation
validation_pool = Pool(data=features_validation_scaled, label=targets_validation_single_col)

# Predict using the trained CatBoost model on the validation set
predictions_validation = catboost_model.predict(validation_pool)

# Since we've converted the validation targets to a single column of labels,
# the predictions from CatBoost are already in the correct format for accuracy calculation
catboost_accuracy_validation = accuracy_score(targets_validation_single_col, predictions_validation.flatten())

print(f"CatBoost Validation set accuracy: {catboost_accuracy_validation:.4f}")

#### Conclusion

`CatBoost` is the best model so far. It reached 91% accuracy without needing a GPU, so we expect our ANN to perform much better.

# 6. Using Keras to train an ANN

We will start from scratch, and use the `Sequential` model from `Keras`.

## 6.1 Define directories

In [ ]:
img_size = (299, 299)
batch_size = 32

In [ ]:
train_dir = p('Training')
valid_dir = p('Testing')

resized_train_dir = o('Resized/Training')
resized_valid_dir = o('Resized/Testing')

# Create the "Resized" directories and subdirectories for each class if they don't exist
os.makedirs(resized_train_dir, exist_ok=True)
os.makedirs(resized_valid_dir, exist_ok=True)

class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Create subdirectories in "Resized" for each class
for class_name in class_names:
    os.makedirs(os.path.join(resized_train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(resized_valid_dir, class_name), exist_ok=True)

## 6.2 Preprocess the data

### Change image format from (w, y, z) to (299, 299, 1)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

def resize_images(image_directory, img_size, savedir):
    # Ensure the base save directory exists
    if not os.path.exists(savedir):
        os.makedirs(savedir)

    for subdir, dirs, files in os.walk(image_directory):
        print(f"Processing directory: {subdir}")
        # Skip the base directory if it doesn't contain images directly
        if not files:
            continue

        current_class = os.path.basename(subdir)  # More portable than splitting on "/"
        print(f"Current class: {current_class}")

        new_savedir = os.path.join(savedir, current_class)

        # Create the class-specific save directory if it doesn't exist
        if not os.path.exists(new_savedir):
            os.makedirs(new_savedir)

        for file in tqdm(files):
            img_path = os.path.join(subdir, file)
            img_savedir = os.path.join(new_savedir, file)

            try:
                img = load_img(img_path)
                img_array = img_to_array(img)
                # Select the first channel if there are several
                if img_array.shape[-1] > 1:
                    img_array = img_array[..., :1]
                # Convert array back to image
                img = array_to_img(img_array, scale=False)
                # Resize the image
                img = img.resize(img_size)
                # Save the resized image
                img.save(img_savedir)
            except Exception as e:
                print(f"Error processing {img_path}: {e}")

resize_images(train_dir, img_size, o("Resized/Training"))
resize_images(valid_dir, img_size, o("Resized/Testing"))


train_generator = train_datagen.flow_from_directory(
    resized_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'  # Use grayscale since we're resizing to a single channel
)

valid_generator = valid_datagen.flow_from_directory(
    resized_valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

### Verifying all images are (299, 299, 1) now

In [ ]:
def verify_size():
    tr_glioma = Collection(directory_path=o("Resized/Training/glioma"), name=p("Glioma (training)"), verbose=1)
    tr_meningioma = Collection(directory_path=o("Resized/Training/meningioma"), name=p("Meningioma (training)"), verbose=1)
    tr_notumor = Collection(directory_path=o("Resized/Training/notumor"), name=p("No tumor (training)"), verbose=1)
    tr_pituitary = Collection(directory_path=o("Resized/Training/pituitary"), name=p("Pituitary (training)"), verbose=1)

    te_glioma = Collection(directory_path=o("Resized/Testing/glioma"), name=p("Glioma (testing)"), verbose=1)
    te_meningioma = Collection(directory_path=o("Resized/Testing/meningioma"), name=p("Meningioma (testing)"), verbose=1)
    te_notumor = Collection(directory_path=o("Resized/Testing/notumor"), name=p("No tumor (testing)"), verbose=1)
    te_pituitary = Collection(directory_path=o("Resized/Testing/pituitary"), name=p("Pituitary (testing)"), verbose=1)

    print("Training and testing sets have been successfully loaded.")

    training_collections = [tr_glioma, tr_meningioma, tr_notumor, tr_pituitary]
    testing_collections = [te_glioma, te_meningioma, te_notumor, te_pituitary]
    all_collections = training_collections + testing_collections

    del tr_glioma, tr_meningioma, tr_notumor, tr_pituitary
    del te_glioma, te_meningioma, te_notumor, te_pituitary

    # Create a header for the printing table
    print(f"{'Collection Name':<25} {'Content Type':<10} {'Format':<20}")
    print('-' * 60)  # Print a dividing line for headers

    # Loop through each collection and print the details in a formatted row
    for col in all_collections:
        col.check_dimension_consistency()
        col.check_image_format_consistency()

        # Determine content type
        if col.dimension_consistency and not col.iscolor:
            content_type = "Fully Grayscale"
        elif col.dimension_consistency and col.iscolor:
            content_type = "Fully Colored"
        else:
            content_type = "Mixed"

        # Determine format consistency
        format_consistency = f"Unique Shape {col.format}" if col.format_consistency else "Different Shapes"

        # Print the collection details in a formatted row
        print(f"{col.name:<25} {content_type:<10} {format_consistency:<20}")

    del training_collections, testing_collections, all_collections
    

# verify_size() <-- in comment because takes a lot of memory

In [ ]:
gc.collect()

## 6.3 Build the Convolutional Neural Network

### First idea: self building the CNN layers

#### Architechture of "model1"

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F17037041%2Fe5b1ba0658d1dea1264783608c206e7c%2Fcnn.png?generation=1712668296423150&alt=media)

### Second idea: use the XCeption pre-built model

#### Architechture of "model2"

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F17037041%2F68f246630994692a95787639af879adc%2FNeural4.png?generation=1709490059509918&alt=media))

#### Architechture of XCeption

![](https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F17037041%2F7f742589a66b82a394b78f4f257e8d7c%2Fxception.png?generation=1712668344529154&alt=media)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 1)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(4, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

In [ ]:
img_shape=(img_size[0],img_size[1],1)

duplicate_channels = Lambda(lambda x: tf.tile(x, [1, 1, 1, 3]))
input_layer = Input(shape=img_shape)
x = duplicate_channels(input_layer)

# Load the base model, excluding its top layer, designed to handle RGB images
base_model = Xception(include_top=False, weights="imagenet", input_tensor=x, pooling='max')


model2 = Sequential([
    base_model,
    Flatten(),
    Dropout(rate= 0.3),
    Dense(128, activation= 'relu'),
    Dropout(rate= 0.25),
    Dense(4, activation= 'softmax')
])

model2.compile(Adamax(learning_rate= 0.001),
              loss= 'categorical_crossentropy',
              metrics= ['accuracy',
                        Precision(),
                        Recall()])
model2.summary()

## 6.4 Train the model

In [ ]:
gc.collect()

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=60,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size
)

In [ ]:
gc.collect()

In [ ]:
history2 = model2.fit(train_generator,
                 epochs=20,
                 validation_data=valid_generator,
                 shuffle= False)

In [ ]:
def plot_history(model, history):
    # Extracting loss and accuracy for plotting
    training_loss = history.history['loss']
    training_accuracy = history.history['accuracy']
    validation_loss = history.history['val_loss']
    validation_accuracy = history.history['val_accuracy']
    epochs = range(1, len(training_loss) + 1)

    # Plotting training and validation loss
    plt.figure(figsize=(14, 7))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, training_loss, 'bo-', label='Training Loss')
    plt.plot(epochs, validation_loss, 'ro-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plotting training and validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, training_accuracy, 'bo-', label='Training Accuracy')
    plt.plot(epochs, validation_accuracy, 'ro-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()
    

plot_history(model, history)
plot_history(model2, history2)

In [ ]:
def val_acc_for_higher_test_acc(history):
    training_accuracy = history.history['accuracy']
    validation_accuracy = history.history['val_accuracy']
    highest_training_accuracy_epoch = np.argmax(training_accuracy)
    
    return validation_accuracy[highest_training_accuracy_epoch]


accuracies = {
    'Conv2D Val Accuracy': val_acc_for_higher_test_acc(history),
    'Catboost Val Accuracy': catboost_accuracy_validation,
    'XCeption Val Accuracy': val_acc_for_higher_test_acc(history2),
}

# Create a bar plot
plt.figure(figsize=(10, 6))
ax = sns.barplot(x=list(accuracies.keys()), y=list(accuracies.values()))
ax.set(ylim=(0.88, 1.0))

# Adding the text labels on the bars
for p in ax.patches:
    ax.annotate(format(p.get_height(), '.2%'), 
                (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha = 'center', va = 'center', 
                xytext = (0, 4), 
                textcoords = 'offset points')

plt.title(f'Final Model Accuracy')
plt.ylabel('Accuracy')
plt.show()

# Conclusion

The XCeption model achieves a 99.8% validation accuracy, which is much better than any other model developped in this notebook.